In [123]:
import numpy as np 
import pandas as pd 
import nltk         ## for text
import re           ## regular expression
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer  ## it converts the word into its base form
## Dataset
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
import networkx as nx

from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

df = pd.read_csv("C:/Users/yashu/Downloads/Telegram Desktop/25_Projects/ML/Text Summarization(Medium)/tennis.csv")
df['article_text'][0]
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\yashu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#### Removing Stopwords

In [126]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yashu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
# initialize stop words
stop_words = stopwords.words('english')
print(stop_words)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [186]:
# removing the stop words from a given sentence
def remove_stopwords(sen):
    new_sen = " ".join([i for i in sen if i not in stop_words])
    #print(new_sen)
    return new_sen

In [188]:
z = []
x = ['maria', 'is', 'a', 'good', 'girl']
z.append(remove_stopwords(x))
print(z)

['maria good girl']


In [236]:
# Loop Through Each Article in the DataFrame
for idx,article in enumerate(df['article_text']):
    #print(idx , article)

    # 1. TOKENIZING THE SENTENCES : converting each text(many sentences) of the para into a separate sentences
    sentence = sent_tokenize(article)
    #print(sentence)

    # 2. Clean the data : remove everything other than aplhabets , convert into smaller aplhabets and remove all the stop words
    
    # 2.1 remove everything other than aplhabets
    clean_sentences = pd.Series(sentence).str.replace("[^a-zA-Z]", " " , regex=True)
    #print(clean_sentences)
    
    # 2.2 convert into smaller aplhabets
    clean_sentences = [s.lower() for s in clean_sentences]
    #print(clean_sentences)

    ######################################## FOR UNDESTANDING ONLY
    # for s in clean_sentences:
    #       print(s)
    ########################################

    #2.3 remove all the stop words
    clean_sentences = [remove_stopwords(s.split()) for s in clean_sentences]
    #print(clean_sentences)

    # 3. **Create Sentence Vectors**  : to convert text into number for system to read
    w_d = {}
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([w_d.get(w, np.zeros((100,))) for w in i.split()]) / (len(i.split()) + 0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
        #print(sentence_vectors)
        
    # 4. **Create Similarity Matrix**  : just 2 check similarities between sentences
    sim_mat = np.zeros([len(sentence) , len(sentence)])
    for i in range(len(sentence)):
        for j in range(len(sentence)):
            if i!= j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                       sentence_vectors[j].reshape(1,100))[0,0]
                #print(sim_mat[i][j])

    # 5. **Build Graph and Apply PageRank**
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #print(scores)
    
    # 6. **Rank Sentences**
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentence)), reverse=True)
    print(ranked_sentences)

        # 7. **Print Article and Summary**
    print(f"ARTICLE {idx + 1}:")
    print(article)
    print('\n')
    print("SUMMARY:")
    for i in range(5):  # Print Top 5 Sentences as Summary
        print(ranked_sentences[i][1])
    print('\n' + '-' * 100 + '\n')


    
    
    
    
    

[(0.058823529411764705, 'When she said she is not really close to a lot of players, is that something strategic that she is doing?'), (0.058823529411764705, "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."), (0.058823529411764705, "Uhm, I'm not really friendly or close to many players."), (0.058823529411764705, "There are so many other things that we're interested in, that we do.'"), (0.058823529411764705, "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much."), (0.058823529411764705, 'Maria Sharapova has basically no friends as tennis players on the WTA Tour.'), (0.058823529411764705, "Is it different on the men's tour than the women's